---
title: 6.4 The QR Algorithm for Eigenvalues
subject:  Eigenvalues
subtitle: 
short_title: 6.4 The QR Algorithm for Eigenvalues
authors:
  - name: Nikolai Matni
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nmatni@seas.upenn.edu
license: CC-BY-4.0
keywords: Eigenvalues, Eigenvectors
math:
  '\vv': '\mathbf{#1}'
  '\bm': '\begin{bmatrix}'
  '\em': '\end{bmatrix}'
  '\R': '\mathbb{R}'
---

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath-/05_Ch_6_Eigenvalues_and_Eigenvectors/074-qr_algorithm.ipynb)

{doc}`Lecture notes <../lecture_notes/Lecture 11 - Eigvenvalues and Eigenvectors part 1 (dynamical systems, determinants, basic definitions and computations).pdf>`

## Reading

Material related to this page, as well as additional exercises, can be found in ALA 8.1.

## Learning Objectives

By the end of this page, you should know:
- the definition of similar matrices,
- the definition of the Schur decomposition of a matrix,
- how to use the QR algorithm to find the eigenvalues of a matrix.

## Motivations

Before we introduce the QR algorithm, we introduce a few motivating definitions.

### Similar Matrices

First, we introduce the notion of *similar matrices*:

:::{prf:definition} Similar matrices
:label: similar-matrices-defn

We say matrices $n\times n$ matrices $A$ and $B$ are similar if $B = P^{-1}AP$ for some invertible matrix $P$.
:::

A key property of similar matrices is that they have the same eigenvalues:

:::{prf:theorem} Eigenvalues of similar matrices
:label: similar-matrices-thm

Suppose $A$ and $B$ are similar. Then, $\lambda$ is an eigenvalue of $A$ if and only if $\lambda$ is an eigenvalue of $B$.
:::

To see why this is true, suppose that $\lambda$ is an eigenvalue of $B$, i.e., $B\vv v = \lambda \vv v$ for some nonzero $\vv v$. Then, 

\begin{align*}
    B \vv v = \lambda \vv v &\implies (P^{-1}AP) \vv v = \lambda \vv v\\
    &\implies A(P \vv v) = P (\lambda \vv v) = \lambda (P\vv v)
\end{align*}

meaning that $P\vv v$ is an eigenvector of $A$, with the same eigenvalue of $\lambda$. 

This motivates the following method: if we want to find the eigenvalues of $B$, we could try to write it in the form $A = P^{-1}BP$, where $B$ is a matrix for which it is easy to find the eigenvalues! This is exactly the motivation for the QR algorithm, which is an iterative algorithm for finding the *Schur decomposition* of $A$.

We will revisit similar matrices in more detail a few sections down the line.

### The (Real) Schur Decomposition

Next, we define the Schur decomposition. The proof of this claim relies on material covered later in the course, so we won't prove it here.

:::{prf:theorem} The real Schur decomposition
:label: real-schur-decomposition-thm

Every real square matrix $A$ can be written in the form:

\begin{align*}
    A = Q^\top U Q.
\end{align*}

where $Q$ is a real orthogonal matrix and $U$ is a real quasi-upper triangular matrix. This is known as a *real Schur decomposition* of $A$, and in general is not unique.

A quasi-upper triangular matrix is a special type of block upper triangular matrix, in which the diagonal blocks are all $1\times 1$ or $2\times 2$.
:::

Note that, given a real Schur decomposition $A = Q^\top U Q$, it's easy to find the eigenvalues of $A$; we just find the eigenvalues of each diagonal block! To see why, recall [Determinant Fact 4](#determinant-properties-defn). Since $U$ is quasi-upper triangular, its diagonal blocks are all $1\times 1$ or $2\times 2$ matrices, which have easy to solve characteristic equations, so it's easy to find their eigenvalues.

## The QR Algorithm

Equipped with these facts, the motivation behind the QR algorithm is as follows. 

* First, we find an *approximation* to a Schur decomposition $A = Q^\top U Q$ (where $Q$ is orthogonal, $U$ is quasi-upper triangular).

* The eigenvalues of $A$ are thus the diagonal entries of $U$.

### Approximating a Real Schur Decomposition

The question thus becomes, how do we find an approximate Schur decomposition for $A$? One idea is repeatedly apply the QR decomposition to iteratively find similar matrices to $A$. In the case that all eigenvalues are distinct, this actually converges to the Schur decomposition.

1. Initialize:
   $$A_1 = A$$

2. For $t = 1, 2, \dots$, perform the following steps:

   a. Compute the QR-factorization of $A_t$:
      $$Q_t R_t = A_t$$
      where $Q_t$ is orthogonal and $R_t$ is upper triangular.

   b. Define $A_{t+1}$ by reversing the factors:
      $$A_{t+1} = R_t Q_t = (Q_t^\top A_t) Q_t$$

The mechanism of why this algorithm works can be described by two phenomena. First, each subsequent $A_{t+1}$ is similar to the previous $A_t$ so we know that eigenvalues are preserved over each iteration. Second, in each interation the elements below the diagonal get smaller and vice versa for those above (showing why this property holds is beyond the scope of this course). This means that over many iterations the elements below the diagonal effectively become 0, making $A_\infty$ a upper-triangular matrix.

## Pseudocode for the QR Algorithm

We are now ready to give the pseudocode for the QR Algorithm.

:::{prf:algorithm} The QR Algorithm for Eigenvalues
:label: qr-eigen-alg

**Inputs** $n\times n$ matrix $A$ with distinct absolute value eigenvalues $\lambda_1, \dots, \lambda_n$; number of iterations $T$

**Output** approximate values of to $\lambda_1, \dots, \lambda_n$

$A_1 \gets A$\
**for** $t = 1$ to $T$:\
$\quad$ $Q_t, R_t \gets $ QR factorization of $A_t$\
$\quad$ $A_{t + 1} \gets R_t Q_t$ \
**return** the eigenvalues of $A_{T + 1}$
:::

## Python Implementations of the QR Algorithm

We are now ready to use the QR Algorithm to find eigenvalues in Python. 

### From Scratch

First, we'll give an implementation of the Schur decomposition from scratch. We won't reimplement a QR factorization algorithm, so if you're interested in seeing the implementation for that, it can be found [on this page (scroll down)](#qr-alg).

In [2]:
import numpy as np

def schur_form(A, iters=100): # A is a square matrix
    B = A
    for i in range(iters):
        Q, R = np.linalg.qr(B)
        B = R @ Q
    return B

print(np.round(schur_form(np.array([
    [2.0, 3.0, 1.0, 0.5, 4.0],
    [4.0, 5.0, 7.0, 0.1, 1.0],
    [5.0, 3.0, 6.0, 19.2, 9.0],
    [1.0, 4.0, 1.0, 4.0, 7.0],
    [3.0, 1.0, 6.0, 2.0, 6.0]
])), 2))

[[21.36 -6.11  9.82  4.82 -2.86]
 [ 0.   -2.14  8.72  3.1  -6.6 ]
 [-0.   -5.12 -1.82  1.99 -0.84]
 [ 0.   -0.    0.    4.34 -0.32]
 [ 0.   -0.    0.    0.    1.26]]


### Using `numpy.linalg.eigvals`

Next, we'll use the `numpy.linalg.eigvals` function, which returns the eigenvalues of a square matrix. Under the hood, `numpy.linalg.eigvals` is based off of a more advanced version of the basic QR algorithm we outlined above. 

In [1]:
import numpy as np

print(np.round(np.linalg.eigvals(np.array([
    [2.0, 3.0, 1.0, 0.5, 4.0],
    [4.0, 5.0, 7.0, 0.1, 1.0],
    [5.0, 3.0, 6.0, 19.2, 9.0],
    [1.0, 4.0, 1.0, 4.0, 7.0],
    [3.0, 1.0, 6.0, 2.0, 6.0]
])).T, 2))

[21.36+0.j   -1.98+6.68j -1.98-6.68j  1.26+0.j    4.34+0.j  ]


Verify for yourself that the real eigenvalues returned by `numpy.linalg.eigvals` are the diagonal $1\times 1$ blocks of the Schur form of the input matrix, which we found above. You can also verify that the complex eigenvalues returned by `numpy.linalg.eigvals` are the eigenvalues of the $2\times 2$ block diagonal.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/nikolaimatni/ese-2030/HEAD?labpath-/05_Ch_6_Eigenvalues_and_Eigenvectors/074-qr_algorithm.ipynb)